In [1]:
# Reference1: Data and Pie chart plotting => https://www.codemag.com/Article/2111061/Building-Dashboards-Using-Bokeh
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.palettes import Category20c
from bokeh.palettes import Spectral9
from bokeh.transform import cumsum
import pandas as pd
import random
import math
import numpy as np

datapath = 'datapandas/'  # data location under pycharm root directory
file = 'ABS2021StateAgegrpIntPopulation.xlsx' #excel input file; state by age group population, columns: State, AgeGroup, Population
output_file('codeflask/templates/populationByState.html')

df = pd.read_excel(datapath+file)
#print(df)
#   State           AgeGroup  Population
#0    NSW          0-9 years      968874
#1    NSW        10-24 years     1455220
#2    NSW        25-39 years     1722217
#3    NSW        40-54 years     1539927
#4    NSW        55-74 years     1750502
#5    NSW  75 years and over      635420
#6    VIC          0-9 years      778082
#7    VIC        10-24 years     1164934
#...

# Group and sum population by State
df = df.groupby('State')[['Population']].sum().reset_index()
# Add a row to where the values of NT and OTH will be added together (OTH has very low value and has almost zero percentage)
df.loc[len(df.index)] = ['NT+OTH', 999]
df.loc[9,'Population'] = df.loc[2,'Population'] + df.loc[3,'Population']
# Drop the individual rows for NT and OTH
df = df.drop(labels=[2,3], axis=0)
#print(df)

# Add 'colors' column at the end using colors from palette 'Category20c'
df['colors'] =  Category20c[len(df['State'])]  #Spectral9
#print(df)

# Add angles column using percentage 'Population' over 'total Population' and convert it to radians by multiplying with 2*math.pi
df['angles'] = df['Population'] / df['Population'].sum() * 2*math.pi
#print(df,"\n")

# Reference2: Bokeh Pie Chart with Custom Labels (adding "%" in wedge/pie chart) => https://www.youtube.com/watch?v=0jZrnBS6laY
value = df['Population'].values
#print(value)

df['cummAngle'] = [(sum(value[0:i+1]) - (item/2))/sum(value) * 2*math.pi for i, item in enumerate(value)]
df['percent'] = df['Population'].apply(lambda x: (x/df['Population'].sum())*100)
df['label'] = ['{:.2f}%'.format(val) for val in df['percent']]
df['cos'] = np.cos(df['cummAngle']) * 1.45  #1.2
df['sin'] = np.sin(df['cummAngle']) * .6   #.6
#print(df,"\n")

source = ColumnDataSource(data=df)

p = figure(width=700,
           height=400,
           title="Population by State as at 2021 Census",
           background_fill_color="#292929",
           border_fill_color = "black",
           outline_line_color="white",
           tools="hover",
           tooltips="@State: @Population{0,0}",
           x_range=source.data['State'])
#print("x-range=", source.data['State'])

p.wedge(x=len(df['State'])/2,
        y=0,
        radius=2.1,
        start_angle=cumsum('angles', include_zero=True),
        end_angle=cumsum('angles'),
        line_color="white",
        fill_color='colors',
        legend_field='State',
        source=source)

p.axis.visible = False
#p.axis.axis_label=None
#p.grid.grid_line_color = None
# Chart title
p.title.text_color = "#e0e0e0"

# Reference 3 - Labelset reference: https://docs.bokeh.org/en/latest/docs/reference/models/annotations.html#bokeh.models.LabelSet
labels = LabelSet(x = 'cos',
                  y ='sin',
                  text = 'label',
                  y_offset = 0,
                  x_offset = 350, # degrees
                  source = source,
                  text_color='Black',
                  text_align = 'center')

p.add_layout(labels)

#show(p)
save(p)




NameError: name 'save' is not defined